# LSA & LDA 실습

In [1]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print('=3')

=3


NLTK 데이터셋을 다운로드하지 않은 상태라면 아래의 커맨드를 통해 다운로드해준다. NLTK는 데이터셋을 다운로드해 주지 않으면 NLTK의 도구들이 제대로 동작하지 않는다.

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 데이터 다운로드 및 확인
실습을 위해 데이터를 다운로드

In [3]:
import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7fb19a69aaf0>)

다운로드한 데이터를 데이터프레임에 저장하고 전체 샘플의 수를 출력한다.

In [4]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

약 108만 개의 샘플이 존재한다. 5개의 샘플만 출력해본다.

In [5]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


publish_date는 이번 실습에 불필요하므로 headline_text만 별도로 저장한다.

In [6]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


데이터에 중복이 있는지 확인해 본다.

In [7]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

약 108만 개의 샘플 중 중복을 제외하면 약 105만 개의 샘플이 존재하는데 이는 약 3만 개의 샘플이 중복 샘플임을 의미한다.
중복 샘플을 제거해준다.

In [8]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

## 데이터 정제 및 정규화
이제 텍스트 데이터를 정제 및 정규화하는 과정을 진행해 본다.
우선 NLTK의 토크나이저를 이용해 전체 텍스트 데이터에 대해서 단어 토큰화를 수행하고, NLTK가 제공하는 불용어 리스트를 사용하여 불용어를 제거한다.

In [9]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


이제 동일한 단어지만 다른 표현을 가지는 단어들을 하나의 단어로 통합(lemmatization)하는 단어 정규화 과정, 그리고 길이가 1 ~ 2인 단어를 제거하는 전처리를 진행한다.

In [10]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


## 역토큰화 및 DTM 생성
DTM을 생성하는 CountVectorizer 또는 TF-IDF 행렬을 생성하는 TfidfVectorizer의 입력으로 사용하기 위해서 토큰화 과정을 역으로 되돌리는 역토큰화(detokenization) 를 수행한다.

In [11]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc

전처리 최종 결과는 train_data에 저장했다. 5개의 샘플을 출력해보자.

In [12]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

전처리 최종 결과인 train_data는 다음에 배울 LDA 실습에서도 재사용 할 것이다. CountVectorizer를 사용하여 DTM을 생성해보자. 단어의 수는 5,000개로 제한한다.

In [13]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

DTM을 생성했고 크기를 확인해보자.
DTM의 크기(shape)는 (문서의 수 × 단어 집합의 크기)이다.

In [14]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


## scikit-learn TruncatedSVD 활용
이제 Truncated SVD를 통해 LSA를 수행해 보자. 토픽의 수를 10으로 정한다. 이는 앞서 배운 하이퍼파라미터 k에 해당되며 행렬 Vk^T가 k × (단어의 수)의 크기를 가지도록 DTM에 TruncatedSVD를 수행한다.

In [15]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 0.01204626, -0.00371201,  0.01826968, ...,  0.00497032,
         0.00142558,  0.0149266 ],
       [ 0.02907276, -0.01096771,  0.01817933, ..., -0.0028364 ,
        -0.01029897, -0.00332909],
       [ 0.00503577, -0.00203489,  0.00974656, ..., -0.00240685,
         0.0012539 ,  0.00374946],
       ...,
       [ 0.02971409,  0.00417502,  0.0248411 , ...,  0.03553603,
         0.01972693,  0.01455705],
       [ 0.06174812, -0.00451568,  0.13664588, ...,  0.8418204 ,
         0.85542486, -0.4074625 ],
       [ 0.07149276,  0.02849891,  0.00177157, ...,  0.01348028,
        -0.02249333,  0.02742473]])

TruncatedSVD를 통해 얻은 행렬 Vk^T 크기를 확인해본다.

In [16]:
print(lsa_model.components_.shape)

(10, 5000)


행렬 Vk^T가 k × (단어의 수) 의 크기를 가지는 것을 확인할 수 있다. 이제 각 행을 전체 코퍼스의 k개의 주제(topic)로 판단하고 각 주제에서 n개씩 단어를 출력해 보자.

In [17]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74635), ('man', 0.4535), ('charge', 0.21086), ('new', 0.14089), ('court', 0.11154)]
Topic 2: [('man', 0.69421), ('charge', 0.30043), ('court', 0.16825), ('face', 0.11419), ('murder', 0.10645)]
Topic 3: [('new', 0.83688), ('plan', 0.23605), ('say', 0.18256), ('govt', 0.1104), ('council', 0.11023)]
Topic 4: [('say', 0.73929), ('plan', 0.35756), ('govt', 0.16704), ('council', 0.13123), ('urge', 0.07507)]
Topic 5: [('plan', 0.73189), ('council', 0.18094), ('govt', 0.13872), ('urge', 0.08701), ('water', 0.07331)]
Topic 6: [('govt', 0.53728), ('court', 0.25548), ('urge', 0.25544), ('fund', 0.21187), ('face', 0.16954)]
Topic 7: [('charge', 0.51854), ('court', 0.45861), ('face', 0.35406), ('plan', 0.12071), ('murder', 0.11342)]
Topic 8: [('win', 0.55317), ('court', 0.35284), ('kill', 0.2081), ('crash', 0.17374), ('australia', 0.09471)]
Topic 9: [('win', 0.58161), ('charge', 0.49313), ('australia', 0.09669), ('cup', 0.07553), ('world', 0.06914)]
Topic 10: [('council', 0.7

# LDA
## TF-IDF 행렬 생성
LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있다. 여기서는 TF-IDF를 사용한다.

TfidfVectorizer를 사용하여 TF-IDF 행렬을 생성해보자. 단어의 수는 5,000개로 제한한다. TF-IDF 행렬을 생성한 후에는 행렬의 크기를 확인한다.

In [18]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


## scikit-learn LDA Model 활용
사이킷런의 LDA 모델을 사용하여 학습한다. LSA와 마찬가지로 동일한 사이킷런 패키지이므로 앞으로 진행되는 실습 과정은 LSA와 매우 유사하다. 토픽의 개수는 10개로 정했는데 이는 n_components의 인자값이다.

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

LDA를 통해 얻은 결과 행렬의 크기를 확인해보자.

In [20]:
print(lda_model.components_.shape)

(10, 5000)


전체 코퍼스로부터 얻은 10개의 토픽과 각 토픽에서의 단어의 비중을 확인해보자.

In [21]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',